# Homework 4: Sentence Segmentation

Sentence segmentation is a crucial, yet under-appreciated, part of any text processing pipeline. In any real-world setting, text will not come to you pre-chunked into sentences, and if your pipeline involves any sentence-level analysis (parsing, translation, entity & co-reference extraction, etc.) the first thing you'll need to do will be split things up into sentences. This is a deceptively challenging problem, as punctuation can be used in multiple ways. Consider the following sentence (borrowed from Kyle Gorman's [excellent discussion of the subject](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/)): 

> _"Rolls-Royce Motor Cars Inc. said it expects its U.S. sales to remain steady at about 1,200 cars in 1990."_ 

This sentence features several periods, but only one of them (the last one) represents a valid sentence break. Sentences can also have embedded clauses or quotation marks, some of which may superficially look like a sentence boundary: 

> _He says the big questions–“Do you really need this much money to put up these investments? Have you told investors what is happening in your sector? What about your track record?–“aren’t asked of companies coming to market._

There are many ways to approach this particular problem; it can be approached using rules, which themselves may be encoded as regular expressions, or (more usefully) it can be modeled as a machine learning problem. Specifically, as a binary classification problem, where candidate sentence boundaries (i.e., punctuation marks from the closed class of English punctuation that can represent a sentence boundary) are either _positive_ examples (actual sentence boundaries) or _false_ (not sentence boundaries). 

In this assignment, you will develop and test your own sentence boundary detection algorithm. This assignment is shorter than our previous assignments, and is also much more open-ended. By this point in the class, you have been exposed to several different families of technique for text classification and analysis, and have hopefully begun to build up some intuitions. For this assignment, you can choose whatever methods you like: log-linear models, rule-based methods, some sort of neural network, it's up to you!

## 0. Setup & data exploration

In [1]:
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [2]:
from hw4_utils import data
from hw4_utils import classifier

We've provided a version of the WSJ section of the Penn Treebank to use for training and evaluation purposes. See `data/wsj_sbd/README.md` for a description of where this data came from and how it was assembled.

The `hw4_utils.data` module has a useful method for reading in data (`file_as_string`) and another utility function to extract candidate boundaries. Let's use these to explore our data a little bit.

In [3]:
dev = data.file_as_string("data/wsj_sbd/dev.txt")
train = data.file_as_string("data/wsj_sbd/train.txt")
test = data.file_as_string("data/wsj_sbd/test.txt")

In [4]:
dev[:100]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is c'

In [5]:
dev[:200]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is chairman of Elsevier N.V., the Dutch publishing group.\nRudolph Agnew, 55 years old and former chairma'

Each of these is a string containing the text, with one input sentence per line. Note that newline characters are still present, which will be important as this will provide us with our ground truth.

Remember, we _only_ will use the `test` split for our final evaluation. For data exploration, feature engineering, and for tuning hyperparameters, use the `dev` split.

The first and most important thing to find out when doing classification is what your class probabilities are, so let's look into that. The `load_candidates()` function in the `hw4_utils.data` module will read an input data split, identify candidate boundaries, and extract information about the candidates context that we may want to use for feature engineering. One of the attributes it will extract is whether or not the candidate is actually a true sentence break (`is_true_break`). Obviously, this would only be useful at training time, as in the "real world" we wouldn't know the answer. :-)

We will use `load_candidates()` to answer our question about class balance:

In [6]:
from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

As is often the case, our classes are quite imbalanced! Let's load in our ground truth labels:

In [7]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]

What else can we find out about our candidate breaks?

In [8]:
candidates = list(data.load_candidates(dev[:10000])) # for now, just get candidates from the first 10k characters
first_can = candidates[0]

In [9]:
print("original context: ",first_can.orig_obs)
print("punctuation mark: ", first_can.punctuation_mark)
print("token to the left: ", first_can.left_token)
print("token to the right: ", first_can.right_token)

original context:  d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El
punctuation mark:  .
token to the left:  Nov
token to the right:  29


**Note:** Take a look at the the `Observation` namedtuple in `hw4_utils.data` for more details about the various attributes that you can find out about each possible sentence boundary.

## 1. Baselines

We will begin by looking at three different baselines. First, we will 

### Baseline #1: Majority class

Since our data are quite imbalanced, what if we always treat every candidate boundary as though it was a true boundary?

In [52]:
y_hat_baseline = [classifier.baseline_classifier(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_baseline))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1766
           1       0.77      1.00      0.87      5769

    accuracy                           0.77      7535
   macro avg       0.38      0.50      0.43      7535
weighted avg       0.59      0.77      0.66      7535



### Baseline #2: Next-token capitalization

What if we say that a candidate is a boundary if the following token is capitalized?

In [11]:
y_hat_next_tok_cap = [classifier.next_tok_capitalized_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_next_tok_cap))

              precision    recall  f1-score   support

       False       0.60      0.40      0.48      1766
        True       0.83      0.92      0.87      5769

    accuracy                           0.80      7535
   macro avg       0.71      0.66      0.67      7535
weighted avg       0.78      0.80      0.78      7535



### Baseline #3: Punkt

Using the [NLTK implementation](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) of [Punkt (Kiss & Strunk, 2006)](https://www.mitpressjournals.org/doi/pdf/10.1162/coli.2006.32.4.485):

(Note: You'll need to have downloaded the NLTK pre-trained Punkt model for this baseline to work)

In [12]:
y_hat_punkt = [classifier.punkt_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_punkt))

              precision    recall  f1-score   support

       False       0.92      0.43      0.59      1766
        True       0.85      0.99      0.91      5769

    accuracy                           0.86      7535
   macro avg       0.88      0.71      0.75      7535
weighted avg       0.87      0.86      0.84      7535



### Conclusion

None of these are especially inspiring in their performance... can you do better?

## 2. Your turn!

### Part 1: Now it's your turn.

Your mission, in this assignment, is to implement at least two additional approaches. The approaches must be different in terms of the features you choose to use, classification approach, or both, but beyond that limitation you are free to do whatever you like as long as it is your own work (i.e., do not download and use an already-existing sentence boundary detector tool). You may, however, replicate an existing algorithm, though please be very careful not about re-using other people's code. 

For a discussion of features that may prove useful, and of other approaches to this task that you may wish to replicate or build off of, consult the bibliography discussed in [Gorman (2014)](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/); note that the specific sentence boundary detector described on that page uses a small set of highly useful features, but you will want to go beyond simply replicating his system.

***Deliverable***: The code for your ≥ two additional classifiers, along with a brief writeup (≈1 page) of what you built and why you chose those specific approaches and features. 

### Part 2: Evaluation & Error analysis

After building and describing your additional classifiers, evaluate their performance against the `test` partition of the data as shown above. Write a brief summary of their relative performance, and include an _error analysis_. Examine cases where your classifiers failed to predict correctly, or disagreed with one another, and see if you can identify any patterns that might explain where they were going awry.

***Deliverable***: Your written summary, as well as any relevant data tables.

In [13]:
import re
import numpy as np

In [14]:

from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

In [15]:
Counter([c.is_true_break for c in data.load_candidates(test)])

Counter({True: 7531, False: 2205})

In [16]:
Counter([c.is_true_break for c in data.load_candidates(train)])

Counter({True: 34826, False: 9453})

### 1: Regex based approach

In [17]:
months = ['Jan', 'Feb', 'Mar', 'Apr','Jun','Jul','Aug','Sept','Sep','Oct','Nov','Dec']
month_list=[]
for i in months:
    month_list.append(i)
    month_list.append(i.upper())
    month_list.append(i.lower())
title= ['Mr','Mrs','Sr','Dr','Prof','Exec','Capt','Lt']
title = re.compile('|'.join(title))
other_abbr=['St','Apt','Rd','Co','Inc','Ltd','Dept','Corp','Blvd','e.g.','encl','etc','fig','acct.','ft','i.e.','no','hr']
abbrebiation=month_list+other_abbr
alpha_numeric = re.compile("([a-z]|[0-9]).*")
abbrebiation = re.compile('|'.join(abbrebiation))
initials = re.compile('([a-zA-Z]\.)*[a-zA-Z]')

In [18]:
def regex_based_classifier(candidate, test_output=False):
    if title.fullmatch(candidate.left_token) or abbrebiation.match(candidate.left_raw) and alpha_numeric.match(candidate.right_token) or initials.fullmatch(candidate.left_token):
        if test_output and candidate.is_true_break:
            print(candidate.left_token,candidate.orig_obs)
            print("False Negative")
        return False
    else:
        if test_output and not candidate.is_true_break:
            print(candidate.left_token,candidate.orig_obs)
            print("False Positive")
        return True

In [19]:
y_train = [o.is_true_break for o in data.load_candidates(train)]

In [20]:
train_result = [regex_based_classifier(o) for o in data.load_candidates(train)]

In [21]:
print(classification_report(y_train, train_result))

              precision    recall  f1-score   support

       False       0.97      0.65      0.78      9453
        True       0.91      0.99      0.95     34826

    accuracy                           0.92     44279
   macro avg       0.94      0.82      0.86     44279
weighted avg       0.92      0.92      0.91     44279



In [22]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]
dev_result = [regex_based_classifier(o) for o in data.load_candidates(dev)]
print(classification_report(y_dev, dev_result))

              precision    recall  f1-score   support

       False       0.97      0.72      0.83      1766
        True       0.92      0.99      0.96      5769

    accuracy                           0.93      7535
   macro avg       0.95      0.86      0.89      7535
weighted avg       0.93      0.93      0.93      7535



In [23]:
y_test = [o.is_true_break for o in data.load_candidates(test)]
test_results = [regex_based_classifier(o) for o in data.load_candidates(test)]
print(classification_report(y_test, test_results))

              precision    recall  f1-score   support

       False       0.98      0.67      0.80      2205
        True       0.91      1.00      0.95      7531

    accuracy                           0.92      9736
   macro avg       0.95      0.83      0.88      9736
weighted avg       0.93      0.92      0.92      9736



Regex based approach here did perform good. The accuracy score here is 92 which is not bad. However it seems that may be there are lot more abbrebiations in the english dictionary than mentioned here, for which the classifier is not able to classify giving False positive and False negative results.
In this approach I validated the sentence breaker by analysing the left and right of the period and if that list presents in my regex database. If yes then I considered it as not a true breaker else a sentence breaker.

This classifier works similar in all three datasets - train, val and test. So, it seems the F1 score in test dataset is improved than train and val data. Overall my regex classifier works pretty well.

### 2 Approach : Feature extraction approach for the train, dev and test dataset and then using decision tree:


In the second approach I thought of creating my own dataframe of important features which is mentioned in the "http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/" article. So here I took features as listed below:
1: EOS/NEOS 

2: left token

3: right token

4: length of left token <3 ?

5: first character of left token is upper?

6: first character of right token is upper?

7: first character of right token is number?

8: length of right token < 2 ?

9: length of left token  < 2 ?

10: left token is '}' ?

11: right token is '{' ?


Using these features in boundary classification task gave me pretty good accuracy using DecisionTreeClassifier.
The accuracy came as : 89 on dev set and 90 on test set.
However if I compare the result with my regex based classifier, I found regex approach works better than the feature selection.
This may be due to the fact that I have missed some very important feature in my classification.
My precision for both the class came pretty well.

I tried to compare the correlated features with the model's feature importance and it seems like:
The correlation of "Left token < 3" and "Left token capital" are highly negative with the target variable. Therefore, these two features can be important for sentence boundary classification.

The same features are generated by the model also using sklearn feature_importance_ funtion.

In [24]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [25]:
train = list(data.load_candidates(data.file_as_string("data/wsj_sbd/train.txt")))
dev = list(data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt")))
test = list(data.load_candidates(data.file_as_string("data/wsj_sbd/test.txt")))

In [26]:
def create_dataset(catagory):
    new_dataframe=[]
    for o in catagory:
        if o.is_true_break==True:
            
            new_dataframe.append(['EOS',o.left_token,o.right_token,int(len(o.left_token)<3),int((o.left_token[0]).isupper()),
            int((o.right_token[0]).isupper()),int((o.right_token).isnumeric()),int(len(o.right_token)<2),
                            int(len(o.left_token)<2),int(o.right_token[0] == '{'),int(o.left_token[0] == '}'),int((o.right_token)=='``')])
        else:
            #print("int(isinstance(o.right_token,int))",o.right_token,int((o.right_token).isnumeric()))
            new_dataframe.append(['NEOS',o.left_token,o.right_token,int(len(o.left_token)<3),int((o.left_token[0]).isupper()),
            int((o.right_token[0]).isupper()),int((o.right_token).isnumeric()),int(len(o.right_token)<2),
                             int(len(o.left_token)<2),int(o.right_token[0] == '{'),int(o.left_token[0] == '}'),int((o.right_token)=='``')])
    return new_dataframe

In [27]:
train_data=create_dataset(train)
dev_data=create_dataset(dev)
test_data=create_dataset(test)

In [28]:
#Creating the Dataframes for the train,dev and test data
train_df = pd.DataFrame(train_data, columns = ['EOS/NEOS', 'Left','Right','L < 3', 'L_capital','R_capital','R=Num','R < 2','L < 2','R = {','L=}','R=``'])
dev_df = pd.DataFrame(dev_data, columns =     ['EOS/NEOS', 'Left','Right','L < 3', 'L_capital','R_capital','R=Num','R < 2','L < 2','R = {','L=}','R=``'])
test_df = pd.DataFrame(test_data, columns =   ['EOS/NEOS', 'Left','Right','L < 3', 'L_capital','R_capital','R=Num','R < 2','L < 2','R = {','L=}','R=``'])


In [29]:
train_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,EOS,community,Hong,0,0,1,0,0,0,0,0,0
1,EOS,time,The,0,0,1,0,0,0,0,0,0
2,EOS,crash,The,0,0,1,0,0,0,0,0,0
3,EOS,contract,The,0,0,1,0,0,0,0,0,0
4,EOS,week,But,0,0,1,0,0,0,0,0,0


In [30]:
dev_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,NEOS,Nov,29,0,1,0,1,0,0,0,0,0
1,EOS,29,Mr,1,0,1,0,0,0,0,0,0
2,NEOS,Mr,Vinken,1,1,1,0,0,0,0,0,0
3,EOS,group,Rudolph,0,0,1,0,0,0,0,0,0
4,EOS,conglomerate,A,0,0,1,0,1,0,0,0,0


In [31]:
test_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,EOS,ago,``,0,0,0,0,0,0,0,0,1
1,NEOS,Inc,unit,0,1,0,0,0,0,0,0,0
2,EOS,unit,Here,0,0,1,0,0,0,0,0,0
3,EOS,intense,Even,0,0,1,0,0,0,0,0,0
4,EOS,union,Still,0,0,1,0,0,0,0,0,0


In [32]:
train_df["Left"]=train_df.index
train_df["Right"]=train_df.index
train_df=train_df.replace({"EOS":1,"NEOS":0})

In [33]:
dev_df["Left"]=dev_df.index
dev_df["Right"]=dev_df.index
dev_df=dev_df.replace({"EOS":1,"NEOS":0})

In [34]:
test_df["Left"]=test_df.index
test_df["Right"]=test_df.index
test_df=test_df.replace({"EOS":1,"NEOS":0})

In [35]:
train_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,1,0,0,0,0,1,0,0,0,0,0,0
1,1,1,1,0,0,1,0,0,0,0,0,0
2,1,2,2,0,0,1,0,0,0,0,0,0
3,1,3,3,0,0,1,0,0,0,0,0,0
4,1,4,4,0,0,1,0,0,0,0,0,0


In [36]:
dev_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,0,0,0,0,1,0,1,0,0,0,0,0
1,1,1,1,1,0,1,0,0,0,0,0,0
2,0,2,2,1,1,1,0,0,0,0,0,0
3,1,3,3,0,0,1,0,0,0,0,0,0
4,1,4,4,0,0,1,0,1,0,0,0,0


In [37]:
test_df.head(5)

,EOS/NEOS,Left,Right,L < 3,L_capital,R_capital,R=Num,R < 2,L < 2,R = {,L=},R=``
0,1,0,0,0,0,0,0,0,0,0,0,1
1,0,1,1,0,1,0,0,0,0,0,0,0
2,1,2,2,0,0,1,0,0,0,0,0,0
3,1,3,3,0,0,1,0,0,0,0,0,0
4,1,4,4,0,0,1,0,0,0,0,0,0


#### Analyzing the correlation between features with the target column:

In [38]:
corr_matrix=train_df.corr()
print(corr_matrix)

           EOS/NEOS      Left     Right     L < 3  L_capital  R_capital  \
EOS/NEOS   1.000000  0.011717  0.011717 -0.561482  -0.742352   0.368136   
Left       0.011717  1.000000  1.000000 -0.011387  -0.000121  -0.005412   
Right      0.011717  1.000000  1.000000 -0.011387  -0.000121  -0.005412   
L < 3     -0.561482 -0.011387 -0.011387  1.000000   0.427095   0.067988   
L_capital -0.742352 -0.000121 -0.000121  0.427095   1.000000  -0.281282   
R_capital  0.368136 -0.005412 -0.005412  0.067988  -0.281282   1.000000   
R=Num     -0.260864 -0.020033 -0.020033 -0.038513   0.201639  -0.331418   
R < 2     -0.024090  0.000400  0.000400 -0.033453   0.016800  -0.082981   
L < 2     -0.149931  0.007282  0.007282  0.451488   0.089452   0.060649   
R = {     -0.000847  0.001262  0.001262 -0.004079  -0.006389  -0.021957   
L=}        0.004289  0.004280  0.004280  0.019182  -0.005533  -0.003963   
R=``       0.130887  0.005126  0.005126 -0.085952  -0.072319  -0.593529   

              R=Num     

In [39]:
features_rel = corr_matrix['EOS/NEOS']
print(features_rel)


EOS/NEOS     1.000000
Left         0.011717
Right        0.011717
L < 3       -0.561482
L_capital   -0.742352
R_capital    0.368136
R=Num       -0.260864
R < 2       -0.024090
L < 2       -0.149931
R = {       -0.000847
L=}          0.004289
R=``         0.130887
Name: EOS/NEOS, dtype: float64


In [40]:
cor_features = features_rel[abs(features_rel) > 0.2]
print(cor_features)

EOS/NEOS     1.000000
L < 3       -0.561482
L_capital   -0.742352
R_capital    0.368136
R=Num       -0.260864
Name: EOS/NEOS, dtype: float64


As we can see the correlation of "Left token < 3" and "Left token capital" are highly negative with the target variable.
Therefore, these two features can be important for sentence boundary classification.

#### Training the Model

In [41]:
model = DecisionTreeClassifier(max_depth=2,random_state=1)

In [42]:
x_train=train_df.drop('EOS/NEOS',axis=1)
y_train=train_df[['EOS/NEOS']]

In [43]:
model.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

#### Validating on dev set

In [44]:
x_dev=dev_df.drop('EOS/NEOS',axis=1)
y_dev=dev_df[["EOS/NEOS"]]

In [45]:
y_pred=model.predict(x_dev)

In [46]:
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.54      0.69      1766
           1       0.87      0.99      0.93      5769

    accuracy                           0.89      7535
   macro avg       0.92      0.76      0.81      7535
weighted avg       0.90      0.89      0.87      7535



#### Evaluating the feature importance

In [47]:
model.feature_importances_
print(dict(zip(dev_df.columns, model.feature_importances_)))

{'EOS/NEOS': 0.0, 'Left': 0.0, 'Right': 0.1520049061424833, 'L < 3': 0.8460748478543719, 'L_capital': 0.0019202460031448538, 'R_capital': 0.0, 'R=Num': 0.0, 'R < 2': 0.0, 'L < 2': 0.0, 'R = {': 0.0, 'L=}': 0.0}


In [48]:
x_test=test_df.drop('EOS/NEOS',axis=1)
y_test=test_df[["EOS/NEOS"]]

In [49]:
y_pred_test=model.predict(x_test)

In [50]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.96      0.56      0.71      2205
           1       0.89      0.99      0.94      7531

    accuracy                           0.90      9736
   macro avg       0.92      0.78      0.82      9736
weighted avg       0.90      0.90      0.89      9736



The model performed well on the test set using the extracted features with accuracy of 90.
Also the precision for both the classes are pretty well.
Overall both regex based and feature extraction based approach worked well on sentence segmentation task